## <a id='table of contents'>Содержание</a>  
<a href='#part1'>1. ПОСТАНОВКА ЗАДАЧИ</a>     
<a href='#part2'>2. ЗАГРУЗКА И ОПИСАНИЕ ДАННЫХ</a>    
<a href='#part3'>3. ПРЕДОБРАБОТКА ДАННЫХ</a>    
<a href='#part4'>4. ОБУЧЕНИЕ МОДЕЛЕЙ </a>   
<a href='#part5'> 4.1. БЕЗ УЧЕТА ДИСБАЛАНСА КЛАССОВ</a>    
<a href='#part6'>4.2. C УЧЕТОМ ДИСБАЛАНСА КЛАССОВВ</a>     
<a href='#part7'>4.2.1. ОБУЧИМ МОДЕЛИ С УЧЕТОМ ВЕСА</a>     
<a href='#part8'>4.2.2. ПРИМЕНИМ ТЕХНИКУ upsampling </a>     
<a href='#part9'>4.2.3. ПРИМЕНИМ ТЕХНИКУ downsampling </a>     
<a href='#part10'>5. ПРОВЕРКА МОДЕЛИ НА ТЕСТОВЫХ ДАННЫХ </a>     

# <a id='part1'>1. ПОСТАНОВКА ЗАДАЧИ</a>
<a href='#table of contents'>к оглавлению</a>

***Цель:*** Обучить модель классификации, которая будет классифицировать клиентов банка по двум типам (Exited):   
0-лояльные клиенты, 1- лояльные клиенты клиенты.   
***Целевая метрика:*** - F1-мера, ее значение должно быть больше ***0.59***. Дополнительная метрика AUC-ROC.

# <a id='part2'> 2. ЗАГРУЗКА И ОПИСАНИЕ ДАННЫХ</a>
<a href='#table of contents'>к оглавлению</a>

### Признаки ###
***RowNumber*** — индекс строки в данных  
***CustomerId*** — уникальный идентификатор клиента  
***Surname*** — фамилия  
***CreditScore*** — кредитный рейтинг  
***Geography*** — страна проживания  
***Gender*** — пол  
***Age*** — возраст  
***Tenure*** — количество недвижимости у клиента  
***Balance*** — баланс на счёте  
***NumOfProducts*** — количество продуктов банка, используемых клиентом  
***HasCrCard*** — наличие кредитной карты  
***IsActiveMember*** — активность клиента  
***EstimatedSalary*** — предполагаемая зарплата  
### Целевой признак###
***Exited*** — факт ухода клиента  

In [5]:
import pandas 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
 

#df=pd.read_csv('/datasets/Churn.csv', sep=',')
#pd.options.display.float_format = '{:.2f}'.format

In [11]:
df = pd.read_csv('D:\\Data_Science\\Churn.csv', sep=',')

FileNotFoundError: ignored

In [ ]:
display(df.head())

In [ ]:
df.info()

***ВЫВОД 1:*** 
    - Пропуски есть только в столбце Tenure 
    - Некоторые столбцы можно удалить, так как эти данные излишни
    - Существуют столбцы с числовыми данными, но типа Object
    - Категориальные признаки необходимо преобразовать в численные в численные

# <a id='part3'>3. ПРЕДОБРАБОТКА ДАННЫХ </a>
<a href='#table of contents'>к оглавлению</a>

Сделаем названия столбцов строчными

In [ ]:
def snake_case(s):
    """Преобразование строк из camelCase в snake_case."""
    return ''.join('_' + c.lower()
                   if all([i != 0,
                           c.isupper(),
                           s[i - 1].islower()])
                   else c.lower()
                   for i, c in enumerate(s))

In [ ]:
df.columns = map(snake_case, df.columns)
print(df.columns)

Удалим "лишние" признаки 

In [ ]:
df = df.drop(['row_number', 'customer_id', 'surname'], axis=1)
df.info()

Заполним пропуски в столбце tenure

In [ ]:
df['tenure'] = df['tenure'].astype('Int64')
print(df['tenure'].dtypes)
df['tenure'].plot(title='Распределние недвижимости по заемщикам', kind='hist', bins=30, figsize=(20,8))
plt.ylabel("Количесто заемщиков")
plt.xlabel("Количество объектов недвижимости")

Из графика видно, что распределение недвижимости среди заемщиков более-менее равномерное и можно заменить пропуски медианной, но в этом случае в тестовую выборку попадут данные из тренировочной. Лучше заменить пропуски 0, так как пропусков мало.

In [ ]:
df.tenure.fillna(0, inplace=True)
df.info()

Преобразуем категориальные признаки 

In [ ]:
data_ohe = pd.get_dummies(df, drop_first=True)
display(data_ohe.head())

Проверим баланс классов

In [ ]:
class_frequency=df['exited'].value_counts(normalize=2/len(df))
print(class_frequency)
class_frequency.plot(kind='bar')
plt.ylabel("Доля класса")
plt.xlabel("Класс")

***ВЫВОД 2:*** 
    - Для снижения количества признаков мы избавились от двух столбцов 'row_number', 'customer_id' и 'surname'
    - В данных наблюдается дисбаланс классов

# <a id='part4'> 4. ОБУЧЕНИЕ МОДЕЛЕЙ </a>
<a href='#table of contents'>к оглавлению</a>

## <a id='part5'>4.1 БЕЗ УЧЕТА ДИСБАЛАНСА КЛАССОВ</a>

Разделим данные на валидационную, тренировочну и тестовую выборки

In [ ]:
features=data_ohe.drop(['exited'], axis=1)
target=data_ohe['exited']

Получим тестовую выборку

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
features, target, test_size=0.20, random_state=12345)

In [ ]:
print(features_train.shape)
print(target_train.shape)

Получим тренировочную и валидационную выборки

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(
features_train, target_train, test_size=0.25, random_state=12345)

In [ ]:
print(features_train)

Стандартизируем признаки

In [ ]:
numeric = ['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card', 'is_active_member', 'estimated_salary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
display(features_train.head())
display(features_valid.head())

***Обучим модель логистической регрессии***

In [ ]:
%%time

model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
predictions_valid=model.predict(features_valid)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

result_auc_roc=roc_auc_score(target_valid, probabilities_one_valid)
  
result_f1=f1_score(target_valid, predictions_valid)

print()

print("AUC_ROC:", result_auc_roc, "F1_score:", result_f1)

In [ ]:
def learning_model (data_features_train, data_target_train, data_features_valid, data_target_valid, model):
    model.fit(data_features_train, data_target_train)
    predictions_valid=model.predict(data_features_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]

    result_auc_roc=roc_auc_score(data_target_valid, probabilities_one_valid)
    result__f1=f1_score(data_target_valid, predictions_valid)    
    print()
    print("F1_score:", result_f1, "AUC_ROC:", result_auc_roc) 
    return result_f1, result_auc_roc

In [ ]:
model=LogisticRegression(random_state=12345)
learning_model(features_train, target_train, features_valid, target_valid, model)

***Обучим модель дерева решений***

In [ ]:
%%time

best_result = 0
best_auc=0
for depth in range(1, 30):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid) 
    result_f1 = f1_score(target_valid, predictions_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    result_auc_roc=roc_auc_score(target_valid, probabilities_one_valid)
    
    print("max_depth =", depth, ": ", end='')
    print("F1_score", result_f1, "AUC_ROC:", result_auc_roc)
    if result_f1 > best_result:
        best_model = model
        best_result = result_f1
        best_auc=result_auc_roc
print()
print("F1_score наилучшей модели на валидационной выборке:", best_result, "AUC_ROC:", best_auc)

***Обучим модель случайного леса***

Подберем глубину

In [ ]:
%%time

best_result = 0
best_depth = 0
best_auc = 0
for depth in range(1, 30, 1):
    model = RandomForestClassifier(n_estimators=20, max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    predictions_valid=model.predict(features_valid)
    result=f1_score(target_valid, predictions_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    result_auc_roc=roc_auc_score(target_valid, probabilities_one_valid)
    
    print("max_depth", depth, result,  "AUC_ROC:", result_auc_roc)
    if result > best_result:
        best_result = result
        best_depth = depth
        best_auc = result_auc_roc
        
print()
print("F1_score наилучшей модели на валидационной выборке:", best_result, "Максимальная глубина:", 
      best_depth, "AUC_ROC:", best_auc)

Подберем количество деревьев, минимальное число объектов в узле дерева и минимальное число объектов в листьях дерева

%%script false

%%time

best_model = None
best_result = 0
best_est=0
best_min_samples_split=0
best_min_samples_leaf=0
for est in range(1, 100, 1):
    for split in range(2, 5):
        for leaf in range(1, 4):
            model = RandomForestClassifier(random_state=12345, max_depth=25, n_estimators=est, min_samples_split=split, min_samples_leaf=leaf) 
            model.fit(features_train, target_train) 
            predictions_valid = model.predict(features_valid)
            result = f1_score(target_valid, predictions_valid)
            
            probabilities_valid = model.predict_proba(features_valid)
            probabilities_one_valid = probabilities_valid[:, 1]
            result_auc_roc=roc_auc_score(target_valid, probabilities_one_valid)
            
            print("max_est =", est, ": ", "split", split, "leaf", leaf, 'result_valid', result, "AUC_ROC:", result_auc_roc)
            if result > best_result:
                best_model = model
                best_result = result
                best_min_samples_split=split
                best_min_samples_leaf=leaf
                best_est=est
print()
print("F1_score наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est,
      "Минимальное количество примеров для разделения:", best_min_samples_split,
     "Минимальное количество объектов в листе:", best_min_samples_leaf)

In [ ]:

%%time

model = RandomForestClassifier(random_state=12345, max_depth=25, n_estimators=17, min_samples_split=2, min_samples_leaf=1) 
model.fit(features_train, target_train) 
predictions_valid = model.predict(features_valid)
result = f1_score(target_valid, predictions_valid)
            
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
result_auc_roc=roc_auc_score(target_valid, probabilities_one_valid)
                        
print()
print("F1_score наилучшей модели на валидационной выборке:", result, "AUC_ROC:", result_auc_roc)

***ВЫВОД 3:*** 
    - Максимальное значение метрики F1_score равное 0.584 получено для модели случайного леса, при этом для модели дерева
    решений это значение не сильно ниже. Модель логистической регресии значительно отстает по показателям от прочих моделей.
    - Подгонка гиперпараметров таких как минимальное количество примеров для разделения и объектов в листе не помогла добиться    необходимого значения метрики не хватило 0.06 пунтков при этом обучение модели заняло более 6 минут, что в десятки раз      больше чем заняло обучение других моделей
    - Добиться показателей метрики F1_score 0.59 и выше без учета дисбаланса классов не получается в разумные сроки

## <a id='part6'>4.2 C УЧЕТОМ ДИСБАЛАНСА КЛАССОВ</a>
<a href='#table of contents'>к оглавлению</a>

### <a id='part7'>4.2.1 ОБУЧИМ МОДЕЛИ С УЧЕТОМ ВЕСА</a>

***Обучим модель логистической регрессии***

In [ ]:
%%time

model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predictions_valid=model.predict(features_valid)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc_LogReg_Balan=roc_auc_score(target_valid, probabilities_one_valid)
f1_score_LogReg_Balan= f1_score(target_valid, predictions_valid)

print("F1_score наилучшей модели на валидационной выборке:", f1_score_LogReg_Balan, "AUC_ROC:", auc_roc_LogReg_Balan)

***Обучим модель дерева решений***

In [ ]:
%%time

best_result = 0
best_auc=0
for depth in range(1, 30):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight='balanced') 
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid) 
    f1_score_DecTreeClass_balan = f1_score(target_valid, predictions_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_DecTreeClass_balan=roc_auc_score(target_valid, probabilities_one_valid)
    
    print("max_depth =", depth, ": ", end='')
    print("F1_score", f1_score_DecTreeClass_balan, "AUC_ROC:", auc_roc_DecTreeClass_balan)
    if f1_score_DecTreeClass_balan > best_result:
        best_result = f1_score_DecTreeClass_balan
        best_auc = auc_roc_DecTreeClass_balan
        
print()
print("F1_score наилучшей модели на валидационной выборке:", 
      best_result, "AUC_ROC:", best_auc)

***Обучим модель случайного леса***

In [ ]:
%%time

model = RandomForestClassifier(random_state=12345, max_depth=25, n_estimators=17, 
                               class_weight='balanced') 
model.fit(features_train, target_train) 
predictions_valid = model.predict(features_valid)
f1_score_RanForClas_balan = f1_score(target_valid, predictions_valid)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc_RanForClas_balan=roc_auc_score(target_valid, probabilities_one_valid)

print()
print("F1_score наилучшей модели на валидационной выборке:", 
      f1_score_RanForClas_balan, "AUC_ROC:", auc_roc_RanForClas_balan)

### <a id='part8'>4.2.2 ПРИМЕНИМ ТЕХНИКУ upsampling</a>
<a href='#table of contents'>к оглавлению</a>

Преобразуем данные

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]
    repeat_near=len(features_zeros)/len(features_ones)
    print("features_zeros", features_zeros.shape)
    print("features_ones", features_ones.shape)
    print("target_zeros", target_zeros.shape)
    print("target_ones", target_ones.shape)
    print("repeat_near", repeat_near)

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)
print(features_upsampled.shape)
print(target_upsampled.shape)

***Обучим модель логистической регрессии***

In [ ]:
%%time

model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_upsampled, target_upsampled) 
predicted_valid=model.predict(features_valid)
f1_score_LogReg_UpSam = f1_score(target_valid, predictions_valid)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc_LogReg_UpSam=roc_auc_score(target_valid, probabilities_one_valid)

print()
print("F1_score наилучшей модели на валидационной выборке:", f1_score_LogReg_UpSam, "AUC_ROC:", auc_roc_LogReg_UpSam)

***Обучим модель дерева решений***

In [ ]:
%%time


best_result = 0
best_depth=0
for depth in range(1, 30):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_upsampled, target_upsampled) 
    predictions_valid = model.predict(features_valid) 
    f1_score_DecTreeClass_upsam = f1_score(target_valid, predictions_valid)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_DecTreeClass_upsam=roc_auc_score(target_valid, probabilities_one_valid)
    print("max_depth =", depth, ": ", end='')
    print("F1_score", f1_score_DecTreeClass_upsam, "AUC_ROC:", auc_roc_DecTreeClass_upsam)
    if f1_score_DecTreeClass_upsam > best_result:
        best_depth=depth
        best_result = f1_score_DecTreeClass_upsam
print()
print("F1_score наилучшей модели на валидационной выборке:", best_result, "max_depth =", best_depth)

***Обучим модель случайного леса***

In [ ]:
##### %%time

best_result = 0
best_est=0
best_min_samples_split=0
best_min_samples_leaf=0
max_depth=0
for est in range(1, 40, 5):
    for split in range(2, 5):
        for leaf in range(1, 4):
            for depth in range(1, 30):
                model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est, min_samples_split=split, min_samples_leaf=leaf, class_weight='balanced') 
                model.fit(features_upsampled, target_upsampled) 
                predictions_valid = model.predict(features_valid)
                result = f1_score(target_valid, predictions_valid)
                print("max_est =", est, ": ", "split", split, "leaf", leaf, 'result_valid', result)
                if result > best_result:
                    best_result = result
                    best_min_samples_split=split
                    best_min_samples_leaf=leaf
                    best_est=est
                    max_depth=depth
                    
print()
print("F1_score наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Глубина:", max_depth,
      "Минимальное количество примеров для разделения:", best_min_samples_split,
     "Минимальное количество объектов в листе:", best_min_samples_leaf)

Если поигратся с глубиной, то можно добиться норм результатов на тестовой выборке. В общем это 
переобучение, как я понимаю раз на тестовой выборке результаты хуже чем на валидационной, то 
произошло переобучение

In [ ]:
max_depth=0
best_result=0
best_auc=0
for depth in range(1, 20):
    model = RandomForestClassifier(random_state=12345, max_depth=depth, 
                               n_estimators=31, min_samples_split=2, min_samples_leaf=1, class_weight='balanced') 

    model.fit(features_upsampled, target_upsampled) 
    predictions_valid = model.predict(features_valid)
    f1_score_RanForClas_upsam = f1_score(target_valid, predictions_valid)

    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]

    auc_roc_RanForClas_upsam=roc_auc_score(target_valid, probabilities_one_valid)
    print("F1_score:", f1_score_RanForClas_upsam, "AUC_ROC:", auc_roc_RanForClas_upsam, "Глубина:", max_depth,)
    if f1_score_RanForClas_upsam > best_result:
        best_result = f1_score_RanForClas_upsam
        max_depth=depth
        best_auc=auc_roc_RanForClas_upsam
        
    
print()
print("F1_score наилучшей модели на валидационной выборке:", best_result, 
      "AUC_ROC:", best_auc, "Глубина:", max_depth)

### <a id='part9'>4.2.3 ПРИМЕНИМ ТЕХНИКУ downsampling </a>
<a href='#table of contents'>к оглавлению</a>

Преобразуем данные

In [ ]:
def downsample (features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    frac_near=len(features_ones)/len(features_zeros)
    print("features_zeros", features_zeros.shape)
    print("features_ones", features_ones.shape)
    print("target_zeros", target_zeros.shape)
    print("target_ones", target_ones.shape)
    print("frac_near", frac_near)

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

***Обучим модель логистической регрессии***

In [ ]:
%%time

model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_downsampled, target_downsampled) 
predicted_valid=model.predict(features_valid) ### здесь predicted_valid
result = f1_score(target_valid, predicted_valid) ### а здесь predictions_valid
print()
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc=roc_auc_score(target_valid, probabilities_one_valid)

print("F1_score:", result, "AUC_ROC:", auc_roc)

***Обучим модель дерева решений***

In [ ]:
%%time

best_model = None
best_result = 0
for depth in range(1, 30):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight='balanced') 
    model.fit(features_downsampled, target_downsampled) 
    predictions_valid = model.predict(features_valid) 
    result = f1_score(target_valid, predictions_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc=roc_auc_score(target_valid, probabilities_one_valid)
    
    print("max_depth =", depth, ": ", end='')
    print("F1_score", result, "auc_roc", auc_roc)
    if result > best_result:
        best_model = model
        best_result = result
        
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc=roc_auc_score(target_valid, probabilities_one_valid)

print("F1_score:", best_result)

***Обучим модель случайного леса***

In [ ]:
%%time

model = RandomForestClassifier(random_state=12345, max_depth=25, n_estimators=17, class_weight='balanced') 
model.fit(features_downsampled, target_downsampled) 
predictions_valid = model.predict(features_valid)
result = f1_score(target_valid, predictions_valid)
print()

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc=roc_auc_score(target_valid, probabilities_one_valid)

print("F1_score:", result, "AUC_ROC:", auc_roc)

***ВЫВОД 4:*** 
    - Лучшие показатели показала модель случайного леса после применения к данным техники upsampling с показателями 
    F1_score=0,604 и AUC_ROC=0.851
    - На втором месте модель логистической регресси после техники downsampling с параметрами F1_score: 0.60 
    AUC_ROC: 0.77

Выведем результаты в виде таблицы

In [ ]:
data = {'Модели': ['LogRegr', 'DecTreeClas', 'RanForestClas'], 
        'balan_f1_score': [0.477, 0.558, 0.548],
        'upsam_f1_score': [0.548, 0.558, 0.599],
        'downsam_f1_score': [0.479, 0.545, 0.547],
        'balanc_AUC_ROC': [0.772, 0.808, 0.823],
        'upsam_AUC_ROC': [0.772, 0.808, 0.842],
        'downsam_AUC_ROC': [0.774, 0.809, 0.826]}
table = pd.DataFrame.from_dict(data)
display(table)

# <a id='part10'>5.ПРОВЕРКА МОДЕЛИ НА ТЕСТОВЫХ ДАННЫХ </a>
<a href='#table of contents'>к оглавлению</a>

Проверим модель случайного леса с оптимальными гиперпараметрами

Проведем маштабирование для тестовых данных 

In [ ]:
features_test[numeric] = scaler.transform(features_test[numeric])
display(features_test.head())

In [ ]:
model = RandomForestClassifier(random_state=12345, max_depth=23, n_estimators=31, min_samples_split=2, 
                               min_samples_leaf=1, class_weight='balanced') 

model.fit(features_upsampled, target_upsampled)

predictions_valid = model.predict(features_valid)
result_valid = f1_score(target_valid, predictions_valid)

predictions_test = model.predict(features_test)
result_test = f1_score(target_test, predictions_test)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc=roc_auc_score(target_valid, probabilities_one_valid)

probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]

auc_roc_test=roc_auc_score(target_test, probabilities_one_test)

print("F1_score_valid:", result_valid, "AUC_ROC_valid:", auc_roc)
print("F1_score_test:", result_test, "AUC_ROC_test:", auc_roc_test)

Проверим модель дерева решений

In [ ]:
model = DecisionTreeClassifier(random_state=12345, max_depth=6) 
model.fit(features_upsampled, target_upsampled) 

predictions_valid = model.predict(features_valid) 
predictions_test = model.predict(features_test)

f1_score_DecTreeClass_upsam = f1_score(target_valid, predictions_valid)
f1_score_DecTreeClass_upsam_test = f1_score(target_test, predictions_test)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]

auc_roc_DecTreeClass_upsam=roc_auc_score(target_valid, probabilities_one_valid)
auc_roc_DecTreeClass_upsam_test=roc_auc_score(target_test, probabilities_one_test)

print("F1_score_valid:", f1_score_DecTreeClass_upsam, "AUC_ROC_valid:", auc_roc_DecTreeClass_upsam)
print("F1_score_test:", f1_score_DecTreeClass_upsam_test, "AUC_ROC_test:", auc_roc_DecTreeClass_upsam_test)

***ОСНОВНОЙ ВЫВОД:*** 
    - Лучшая модель - дерева решений, с показателями на тестовой выборке - F1_score_test: 0.593 AUC_ROC_test: 0.818
    - Модель случайного леса показала себя хуже на тестовой выборке, чем на валидационной